<a href="https://colab.research.google.com/github/cimbelli/Inland_Navigable_Waterway/blob/main/s1_danube_1_7_smederevo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Derive the total km of Serbian navigable waterways of rivers and canals in year ...

In [ ]:
#year = '2021'

In [5]:
### comment below after the first run

#!pip install geojson
#!pip install earthengine-api

url = 'https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/smederovo.geojson'
!wget $url


--2022-09-28 15:45:01--  https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/smederovo.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6254 (6.1K) [text/plain]
Saving to: ‘smederovo.geojson’

smederovo.geojson   100%[===================>]   6.11K  --.-KB/s    in 0s      

2022-09-28 15:45:01 (69.0 MB/s) - ‘smederovo.geojson’ saved [6254/6254]



In [2]:
import ee
#import time
import matplotlib
import numpy as np
import requests
#import geemap
#from urllib.request import urlopen
#import urllib

import os
import time
import geojson
import datetime, calendar
import json
import csv

import folium
#import geopandas as gpd

In [6]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
           
def download(url, tipo, name):
    response = requests.get(url)
    
    if tipo == "raster":
        imgref = os.path.join(name + ".tif")
        with open(imgref, 'wb') as fd:
            fd.write(response.content)
    else:
        vec = os.path.join(name + ".geojson")
        with open(vec, 'wb') as fd:
            fd.write(response.content)
            
    return None

def download_raster(img, name):
    url = img.getDownloadUrl({
        'bands': ['VV'],
        'region': bb,
        'scale' : 10,
        'format': 'GEO_TIFF',
        'maxPixels': 834607840,
        'formatOptions': {
            'cloudOptimized': True
          }
    })
    download(url,"raster", name)
    return None

In [7]:
#start_time = time.time()
# Trigger the authentication flow.

#ee.Authenticate()



# Initialize the library.
ee.Initialize()
maxPixels: 6362493000



image_viz_params = {
    'bands': ['VV'],
    'min': 0.0,
    'max': 1.0,
    #'gamma': [0.95, 1.1, 1]
}

In [8]:
f = open('smederovo.geojson')
data1 = json.load(f)

csvfilename = "Serbia_smederovo.csv"
header = ['year', 'month', 'vessels', 'imgs']
with open(csvfilename, 'w', encoding='UTF8') as f1:
    writer = csv.writer(f1)
    writer.writerow(header)

    for year in range (2017, 2022):

        for m in range(1,13):
            
            id1 = data1['features'][0]['properties']['id']
            if id1 == 19:
                coords = data1['features'][0]['geometry']['coordinates']
                try:
                    [j.pop(2) for j in coords[0][0]]
                except:
                  None
                
                aoi = ee.Geometry.Polygon(coords[0][0])

                m1 = ('0' if m <10 else'') + str(m)
                date1 = str(year) + '-' + m1 + '-01'
                date2 = str(year) + '-' + m1 + '-' + str(calendar.monthrange(int(year), m)[1])
                
                #if id1 > 20:
                    #print(id1)

                imgVV = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')   #S1_GRD_FLOAT
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) 
                        .filter(ee.Filter.eq('instrumentMode', 'IW'))
                        .select('VV')
                        .filterDate(date1, date2)
                        .filterBounds(aoi)
                        #.reduce(ee.Reducer.percentile([50]))
                        )

                imgVV_fixed = imgVV.max().clip(aoi).toUint8()
                im = imgVV_fixed

                center = list(aoi.centroid().getInfo().values())[1][1], list(aoi.centroid().getInfo().values())[1][0]

                ####### map
                #map1 = folium.Map(location=center, zoom_start=10)
                #folium.Map.add_ee_layer = add_ee_layer
                #map1.add_ee_layer(im, image_viz_params, 'false color composite')
                #display(map1)

                classes = im.reduceToVectors(
                    **{
                    'reducer': ee.Reducer.countEvery(),
                    'geometry': aoi.getInfo(),
                    'scale': 10,
                    'maxPixels': 100000000
                })

                vessels = (ee.FeatureCollection(classes).filter(ee.Filter.gt('count', 15)).filter(ee.Filter.lt('count', 10000)))
                vessels_centroids = vessels.map(lambda f: f.centroid(10))

                #vessels_centroids1 = vessels_centroids.intersection(aoi)
                vessels_centroids1 = vessels_centroids.filter(ee.Filter.bounds(aoi))
                n_vessels = vessels_centroids1.size().getInfo()
                n_imgs = imgVV.size().getInfo()
                dens = round(100*n_vessels/n_imgs,2)

                print("there are", n_vessels, "vessels", "in the", type1, "with id", id1, "and lenght", lenkm1, "km", "n_img", imgVV.size().getInfo())
                writer.writerow([year, m, n_vessels, n_imgs])
                
                
                downloadUrl1 = vessels_centroids1.getDownloadURL(**{'filetype': 'GEOJSON'})
                download(downloadUrl1, "vessels", "vessels_id_" + str(id1) + "_" + year)
                print(year, m, n_vessels)

print("finito")



EEException: ignored